<a href="https://colab.research.google.com/github/anruki/Quantum-Programming-Applications/blob/main/svm_quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inicialización

Importar librerías

In [1]:
import numpy as np
from sklearn.datasets import load_iris
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

Descarga del dataset:

In [2]:

# Cargar el dataset de Iris
iris = load_iris()

# Convertir a un DataFrame de pandas
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

Utiliza solo los tipos `setosa` y `versicolor`

In [3]:
# Filtrar solo las clases Setosa (0) y Versicolor (1)
df = df[df['target'] != 2]

Mapear los valores de la columna target a los nombres de las especies y mostrar el df

In [4]:
# Mapear los valores de la columna target a los nombres de las especies
# df['target'] = df['target'].map({0: 'setosa', 1: 'versicolor'})

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


Veamos si hay Nan


In [5]:
# Comprobar si hay valores nulos en el DataFrame
nan_counts = df.isnull().sum()
nan_counts

,0
sepal length (cm),0
sepal width (cm),0
petal length (cm),0
petal width (cm),0
target,0


Dividir el dataset en entrenamiento y prueba: Divide el dataset filtrado en conjuntos de entrenamiento y prueba.

In [6]:
from sklearn.model_selection import train_test_split

# Dividir los datos en características (X) y etiquetas (y)
X = df.drop(columns=['target'])
y = df['target']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (80, 4)
Tamaño del conjunto de prueba: (20, 4)


In [7]:
y_train

,target
55,1
88,1
26,0
42,0
69,1
...,...
60,1
71,1
14,0
92,1


In [8]:
X_train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
55,5.7,2.8,4.5,1.3
88,5.6,3.0,4.1,1.3
26,5.0,3.4,1.6,0.4
42,4.4,3.2,1.3,0.2
69,5.6,2.5,3.9,1.1
...,...,...,...,...
60,5.0,2.0,3.5,1.0
71,6.1,2.8,4.0,1.3
14,5.8,4.0,1.2,0.2
92,5.8,2.6,4.0,1.2


## Representación de los datos con statevectors


In [9]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [10]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.1 MB/s eta 0:00:00


In [11]:
X_train.head(1)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
55,5.7,2.8,4.5,1.3


In [12]:
# Crear el escalador
scaler = MinMaxScaler(feature_range=(0, 2 * np.pi))

# Ajustar y transformar los datos
X_train_Scaled = scaler.fit_transform(X_train)

In [13]:
X_train_Scaled[1]

array([3.02523737, 2.61799388, 4.96040945, 5.02654825])

In [14]:
X_train_Scaled[1][0]

3.025237370123504

In [15]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Crear un circuito cuántico con 2 qubits
qc = QuantumCircuit(2)

# Aplicar rotaciones Rx a los qubits
theta_0 = X_train_Scaled[1][0] # Ángulo de rotación para el primer qubit
theta_1 = X_train_Scaled[1][1] # Ángulo de rotación para el primer qubit
theta_2 = X_train_Scaled[1][2] # Ángulo de rotación para el segundo qubit
theta_3 = X_train_Scaled[1][3] # Ángulo de rotación para el segundo qubit

qc.h(0)
qc.h(1)

qc.rx(theta_0, 0)  # Aplicar Rx(theta_0) al qubit 0
qc.ry(theta_1, 0)  # Aplicar Rx(theta_1) al qubit 0

qc.rx(theta_2,  1) # Aplicar Rx(theta_2) al qubit 1
qc.rz(theta_3, 1)  # Aplicar Rx(theta_3) al qubit 1


print(qc)


     ┌───┐┌────────────┐┌──────────┐
q_0: ┤ H ├┤ Rx(3.0252) ├┤ Ry(5π/6) ├
     ├───┤├────────────┤├──────────┤
q_1: ┤ H ├┤ Rx(4.9604) ├┤ Rz(8π/5) ├
     └───┘└────────────┘└──────────┘


In [16]:
pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 67.8 MB/s eta 0:00:00
